# DIARIO

In [113]:
import autograd.numpy as np_
from autograd import grad
import yfinance as yf

df_petr4 = yf.download('PETR4.SA', start='2000-01-01', end='2023-10-22')[['Close']].dropna()

num_passos = 6
X = np_.array([df_petr4['Close'].values[i-num_passos:i] for i in range(num_passos, len(df_petr4))])
Y = np_.array(df_petr4['Close'].values[num_passos:])


[*********************100%***********************]  1 of 1 completed


In [114]:
tam_treino = int(len(X) * 0.75) # pegar 75% para teste
X_train, X_test, y_train, y_test = X[:tam_treino], X[tam_treino:], Y[:tam_treino], Y[tam_treino:]

# normalizando os dados, sem fazer isso da problema na hora da multiplicacao e de encontrar o verdadeiro valor
# preditivo, explico melhor no vídeo, ou seja, ao normalizar estamos fazendo com que nao tenha uma diferenca tao grande
# entre os valores, alem de acelerar o processo de aprendizado!!!
media_X, desv_X = X_train.mean(), X_train.std()
X_train = (X_train - media_X) / desv_X
X_test = (X_test - media_X) / desv_X

# garantir que seja possivel a multiplicacao entre as matrizes no sistema de gradiente descendente, para isso é
# preciso alterar a estrutura que o X_train e X_test possuem para algo factivel de mult
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# coluna de bias (viés), nesse caso não é preciso, entao acrescentamos um valor de 1 para tudo no comeco das matrizes
X_train = np_.c_[np_.ones((X_train.shape[0], 1)), X_train]  
X_test = np_.c_[np_.ones((X_test.shape[0], 1)), X_test]   


In [115]:
num_dimensoes = X_train.shape[1]
w = np_.random.randn(num_dimensoes, 1)
alpha = 0.001 
n_treinos = 1001


# funcao do EQM usada no notebook! tirando o bias
def deqm(w, X, Y):
    y_pred = X @ w
    return np_.mean((y_pred - Y) ** 2)

# funcao para acuracia, peguei do notebook
def acuracia(y_test, y_pred):
    return np_.mean(np_.sign(y_test) == np_.sign(y_pred))


grad_w = grad(lambda w, X, Y: deqm(w, X, Y))

for _ in range(n_treinos):
    dw = grad_w(w, X_train, y_train)
    w -= alpha * dw

y_pred = X_test @ w
eqm = np_.mean((y_pred - y_test) ** 2)
acc = acuracia(y_test, y_pred)


print(f"EQM: {eqm}")
print(f"Acuracia:  {acc}")


EQM: 7.165410343393874
Acuracia:  1.0


# SEMANAL

In [116]:
df_petr4_semanal = df_petr4.resample('W').mean()  

n_passos = 4
X_semanal = np_.array([df_petr4_semanal['Close'].values[i-n_passos:i] for i in range(n_passos, len(df_petr4_semanal))])
Y_semanal = np_.array(df_petr4_semanal['Close'].values[n_passos:])

In [117]:
tam_treino = int(len(X_semanal) * 0.75)

X_train_sem, X_test_sem, y_train_sem, y_test_sem = X_semanal[:tam_treino], X_semanal[tam_treino:], Y_semanal[:tam_treino], Y_semanal[tam_treino:]

media_X_sem, desv_X_sem = X_train_sem.mean(), X_train_sem.std()
X_train_sem = (X_train_sem - media_X_sem) / desv_X_sem
X_test_sem = (X_test_sem - media_X_sem) / desv_X_sem

X_train_sem = X_train_sem.reshape(X_train_sem.shape[0], -1)
X_test_sem = X_test_sem.reshape(X_test_sem.shape[0], -1)

X_train_sem = np_.c_[np_.ones((X_train_sem.shape[0], 1)), X_train_sem]
X_test_sem = np_.c_[np_.ones((X_test_sem.shape[0], 1)), X_test_sem]

In [118]:
num_dimensoes_sem = X_train_sem.shape[1]
w_sem = np_.random.randn(num_dimensoes_sem, 1)  

grad_w_sem = grad(lambda w, X, Y: deqm(w, X, Y))

for i in range(n_treinos):
    dw = grad_w_sem(w_sem, X_train_sem, y_train_sem.reshape(-1, 1))
    w_sem -= alpha * dw

y_pred_sem = X_test_sem @ w_sem
eqm_sem = np_.mean((y_pred_sem - y_test_sem.reshape(-1, 1)) ** 2)
acc_sem = acuracia(y_test_sem.reshape(-1, 1), y_pred_sem)

print(f"EQM semanal: {eqm_sem}")
print(f"Acuracia semanal: {acc_sem}")


EQM semanal: 10.998253402841009
Acuracia semanal: 1.0


# MENSAL

In [119]:
df_petr4_mensal = df_petr4.resample('ME').mean()  

n_passos = 4
X_mensal = np_.array([df_petr4_mensal['Close'].values[i-n_passos:i] for i in range(n_passos, len(df_petr4_mensal))])
Y_mensal = np_.array(df_petr4_mensal['Close'].values[n_passos:])

In [120]:
tam_treino_mensal = int(len(X_mensal) * 0.75)

X_train_mensal, X_test_mensal, y_train_mensal, y_test_mensal = X_mensal[:tam_treino_mensal], X_mensal[tam_treino_mensal:], Y_mensal[:tam_treino_mensal], Y_mensal[tam_treino_mensal:]

media_X_mensal, desv_X_mensal = X_train_mensal.mean(axis=0), X_train_mensal.std(axis=0)
X_train_mensal = (X_train_mensal - media_X_mensal) / desv_X_mensal
X_test_mensal = (X_test_mensal - media_X_mensal) / desv_X_mensal

X_train_mensal = X_train_mensal.reshape(X_train_mensal.shape[0], -1)
X_test_mensal = X_test_mensal.reshape(X_test_mensal.shape[0], -1)

X_train_mensal = np_.c_[np_.ones((X_train_mensal.shape[0], 1)), X_train_mensal]
X_test_mensal = np_.c_[np_.ones((X_test_mensal.shape[0], 1)), X_test_mensal]

In [121]:
num_dimensoes_mensal = X_train_mensal.shape[1]
w_mensal = np_.random.randn(num_dimensoes_mensal, 1)  

grad_w_mensal = grad(lambda w, X, Y: deqm(w, X, Y))

for i in range(n_treinos):
    dw = grad_w_mensal(w_mensal, X_train_mensal, y_train_mensal)
    w_mensal -= alpha * dw

y_pred_mensal = X_test_mensal @ w_mensal
eqm_mensal = np_.mean((y_pred_mensal - y_test_mensal) ** 2)
acc_mensal = acuracia(y_pred_mensal, y_test_mensal)

print(f"EQM mensal: {eqm_mensal}")
print(f"Acuracia mensal: {acc_mensal}")


EQM mensal: 21.706437860124726
Acuracia mensal: 1.0


# FAZENDO GRAFICOS

In [122]:
import pandas as pd

def cria_dataframe(df, y_test, y_pred):
    return pd.DataFrame({
        'Data': df.index[-len(y_test):],
        'Real': y_test.flatten(),
        'Previsto': y_pred.flatten()
    })

df_diario = cria_dataframe(df_petr4, y_test, y_pred)
df_semanal = cria_dataframe(df_petr4_semanal, y_test_sem, y_pred_sem)
df_mensal = cria_dataframe(df_petr4_mensal, y_test_mensal, y_test_mensal)

In [123]:
import plotly.graph_objects as go
import pandas as pd

def cria_grafico(df, titulo):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Data'], y=df['Real'], mode='lines', name='real', line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=df['Data'], y=df['Previsto'], mode='lines', name='previsto', line=dict(color='purple', dash='dot')))
    fig.update_layout(title=titulo, xaxis_title="data", yaxis_title="preço")
    return fig

fig_diario = cria_grafico(df_diario, "Escala diaria")
fig_semanal = cria_grafico(df_semanal, "Escala semanal")
fig_mensal = cria_grafico(df_mensal, "Escala mensal")

fig_diario.show()
fig_semanal.show()
fig_mensal.show()
